In [2]:
import os
import sys
import os
from collections import defaultdict
import pandas as pd
from tqdm import tqdm
import csv


src_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if src_path not in sys.path:
    sys.path.append(src_path)


from Util.posttree import PostTree, UserCommentHistories
from LLM.Mistral.minstral_utils import MistralPrompt

/home/bhx5gh/Documents/CBM/CBM_Final_Project/.venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
post_path = '/home/bhx5gh/Documents/CBM/CBM_Final_Project/Data/arcticshift/processed/r_books_posts.csv'
cleaned_and_pruned_comments_path = '/home/bhx5gh/Documents/CBM/CBM_Final_Project/Data/arcticshift/cleaned/r_books_comments_cleaned_and_pruned.csv'
cleaned_comments_path = '/home/bhx5gh/Documents/CBM/CBM_Final_Project/Data/arcticshift/cleaned/r_books_comments_cleaned.csv'

post_ids = pd.read_csv(post_path, quoting=csv.QUOTE_NONNUMERIC,
            escapechar='\\',
            encoding='utf-8')['post_id'].tolist()
posts_to_exclude = ['1atcjtj', '1au5njg', '18w4y9z','18vx8ip', '18wom5l']


In [7]:
prompt_dataset = {
    'prompt': [],
    'response': []
}

for post_id in tqdm(post_ids):
    if post_id in posts_to_exclude:
        continue
    
    post_tree  =post_tree = PostTree(post_path, post_id)
    post_tree.create_comment_tree(cleaned_and_pruned_comments_path)

    nodes = original_nodes = list(post_tree.bfs_generator())
    user_histories = UserCommentHistories(
            cleaned_comments_path,
            post_id
            )
    for node in nodes:
        cur_user_comment_history = user_histories.get_random_user_history(node.user)

        previous_comments = [c.comment_text for c in node.get_previous_responses()]
        comment_of_interest = "" if len(previous_comments) == 0 else previous_comments[-1]
        mistral_prompt = MistralPrompt(
                        post_title=post_tree.title,
                        post_content=post_tree.content,
                        previous_comments_on_thread=previous_comments,
                        comment_of_interest=comment_of_interest,
                        user_comment_history=cur_user_comment_history,
                    )
        prompt = mistral_prompt.zero_shot_prompt()
        response = node.comment_text
        prompt_dataset['prompt'].append(prompt)
        prompt_dataset['response'].append(response)

100%|██████████| 444/444 [16:58<00:00,  2.29s/it]


In [9]:
prompt_dataset_df = pd.DataFrame.from_dict(prompt_dataset)
prompt_dataset_df.to_csv('/home/bhx5gh/Documents/CBM/CBM_Final_Project/Data/instruct_tune_dataset/prompt_dataset.csv',
                        quoting = csv.QUOTE_NONNUMERIC,
                        escapechar = '\\',
                        encoding = 'utf-8')
